In [3]:
%cd ..
from pathlib import Path
import pandas as Pd

DATA_DIR = Path("data")
drivers_Path = DATA_DIR/"drivers.csv"
Lap_Times_path = DATA_DIR/"lap_times.csv"

drivers = Pd.read_csv(drivers_Path)
lap_times = Pd.read_csv(Lap_Times_path)

drv = drivers.copy()
drv["drivers_name"] = drv["forename"].str.strip() + " " + drv["surname"].str.strip()

best_laps = (
    lap_times
    .groupby(["raceId", "driverId"])["milliseconds"]
    .min()
    .reset_index()
    .rename(columns={"milliseconds":"bestLap_ms"})
)

best_laps["bestLaps_s"] = best_laps["bestLap_ms"]/1000.0
best_laps_named = best_laps.merge(drv[["driverId", "drivers_name"]], on="driverId", how="left")

best_laps_named.head(10)

f:\Personal Projects\F1-FastestLap-Predictor


,raceId,driverId,bestLap_ms,bestLaps_s,drivers_name
0,1,1,89020,89.020,Lewis Hamilton
1,1,2,88283,88.283,Nick Heidfeld
2,1,3,87706,87.706,Nico Rosberg
3,1,4,88712,88.712,Fernando Alonso
4,1,6,89923,89.923,Kazuki Nakajima
5,1,7,89823,89.823,Sébastien Bourdais
6,1,8,88488,88.488,Kimi Räikkönen
7,1,9,87988,87.988,Robert Kubica
8,1,10,88416,88.416,Timo Glock
9,1,12,90502,90.502,Nelson Piquet Jr.
